## Algoritmo Genético

No jupyter anterior utilizamos o algoritmo Gradiente Decrescente para otimizar a função objetivo, afim de comparações, dessa vez utilizaremos um algoritmo genético.

In [1]:
from random import randint, uniform, random
from tqdm import tqdm
import numpy as np

In [2]:
# Manipulação das matrizes

def somar(A, B):
    C = []
    nLinhasA, nLinhasB = len(A), len(B)
    nColA, nColB = len(A[0]), len(B[0])
    
    for i in range (nLinhasA):
        linha = [0]*nColA
        C.append(linha)
        for j in range(nColA):
            C[i][j] = A[i][j] + B[i][j]

    return C

def sub(A, B):
    C = []
    nLinhasA, nLinhasB = len(A), len(B)
    nColA, nColB = len(A[0]), len(B[0])
    
    for i in range (nLinhasA):
        linha = [0]*nColA
        C.append(linha)
        for j in range(nColA):
            C[i][j] = A[i][j] - B[i][j]

    return C


#################################################################

## d (psi) / dt = -i * H * |psi>
def dpsidt(t, psi, H): # A derivada da onda em relação ao tempo não tem dependência temporal
    A = np.zeros((2,2), dtype=np.complex_)
    A = np.dot(complex(0,1),H) # i * H
    return -1*np.matmul(A,psi) # - i * H * |psi>

# Runge-Kutta de quarta ordem

def rungeKutta(onda, fatorRungeKutta, hamiltoniano, tempoFinal = 2, tempoInicial = 0):
    
    resultado = np.zeros((2,2), dtype=np.complex_)
    indice = 0

    for i in range(0, 2):
        
        resultado[0][indice] = onda[0][0]
        resultado[1][indice] = onda[1][0]
        
        k1 = dpsidt(tempoInicial, onda, hamiltoniano)
        k2 = dpsidt(tempoInicial + 0.5 * fatorRungeKutta, somar(onda, np.dot((0.5*fatorRungeKutta), k1)), hamiltoniano)
        k3 = dpsidt(tempoInicial + 0.5 * fatorRungeKutta, somar(onda, np.dot((0.5*fatorRungeKutta), k2)), hamiltoniano)
        k4 = dpsidt(tempoInicial + fatorRungeKutta, somar(onda, np.dot(fatorRungeKutta, k3)), hamiltoniano)
        
        ## y(i+1) = y(i) + h/6*(k1+2*k2+2*k3+k4)
        
        A = somar(np.dot(2,k3), k4)
        B = somar(np.dot(2,k2), k1)
        C = somar(A, B)
 
        onda = somar(onda,np.dot((fatorRungeKutta / 6.0),(C)))
    
        tempoInicial += fatorRungeKutta
        
        indice += 1
        
    runge = np.zeros((2,1), dtype=np.complex_)
    runge[0][0] = resultado[0][indice-1]
    runge[1][0] = resultado[1][indice-1]
    return runge

def funcaoObjetivo(x, iteracao, ondaDestino, hamiltoniano, onda, horizonteAnalisavel, fatorRungeKutta, tempoAnalise):
    
    avanco = np.zeros((2,1),dtype=np.complex_)
    
    y = 0
    
    horizonte = (horizonteAnalisavel+iteracao)/100
    
    tempo = iteracao/100
    
    tempoInicialAnalise = 0.00
    tempoFinalAnalise = 0.02
    
    controles = np.zeros((2,2), dtype=np.complex_)
    controles = [[0, x[0]], [x[0], 0]]
    
    matrizOrigem = np.zeros((2,1),dtype=np.complex_)
    matrizOrigem = [[onda[0][0]], [onda[1][0]]] 

    matrizDestino = np.zeros((2,1),dtype=np.complex_)
    
    ## Função-Objetivo (Return) = somatorio ||(Matriz_Origem - Matriz_Destino)||^2
    
    ## Avanço temporal
    
    matrizDestino[0][0] = ondaDestino[0][0]*np.exp(-1*complex(0,1)*(1/2*np.pi)*tempo)
    matrizDestino[1][0] = ondaDestino[1][0]*np.exp(-1*complex(0,1)*(3/2*np.pi)*tempo)

    y += (np.linalg.norm(matrizOrigem-matrizDestino))**2

    avanco = rungeKutta(matrizOrigem, fatorRungeKutta, somar(hamiltoniano, controles), tempoFinal = tempoFinalAnalise, tempoInicial = tempoInicialAnalise)

    matrizOrigem = avanco

    tempoInicialAnalise += 0.01
    tempoFinalAnalise += 0.01
    
    for indice in range((iteracao+1), (horizonteAnalisavel+iteracao)):
        tempo = indice/100
        
        matrizDestino[0][0] = ondaDestino[0][0]*np.exp(-1*complex(0,1)*(1/2*np.pi)*tempo)
        matrizDestino[1][0] = ondaDestino[1][0]*np.exp(-1*complex(0,1)*(3/2*np.pi)*tempo)
        
        y += (np.linalg.norm(matrizOrigem-matrizDestino))**2
        
        controles = [[0, x[int(tempoInicialAnalise*100)]], [x[int(tempoInicialAnalise*100)], 0]]

        avanco = rungeKutta(matrizOrigem, fatorRungeKutta, somar(hamiltoniano, controles), tempoFinal = tempoFinalAnalise, tempoInicial = tempoInicialAnalise)

        matrizOrigem = avanco

        tempoInicialAnalise += 0.01
        tempoFinalAnalise += 0.01
    
    return y

In [3]:
# Parâmetros iniciais

tamanhoPopulacao = 30
taxaMutacao = 0.01
taxaCrossover = 0.85
numeroGeracoes = 70
quantidadeVariaveis = 3 # Para o sistema dinâmico avaliado, apenas os controles devem ser avaliados
quantidadeTrocas = 2 # Posicoes a serem trocadas na 

In [4]:
def gerarPopulacao(xMin, xMax):
    populacao = np.zeros((tamanhoPopulacao,quantidadeVariaveis))
    for individuo in range(tamanhoPopulacao):
        for pos in range(quantidadeVariaveis):
            populacao[individuo,pos] = uniform(xMin, xMax)
    return populacao

In [5]:
def divisaoFitness(valorFuncaoObjetivo):
    divisao = 0
    for i in range(len(valorFuncaoObjetivo)):
        divisao += valorFuncaoObjetivo[i]
    return divisao

def fitness(valorFuncaoObjetivo):
    return valorFuncaoObjetivo/divisaoFitness(valorFuncaoObjetivo)

In [6]:
def calcularFuncao(populacao, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise):
    valorFuncaoObj = np.zeros(tamanhoPopulacao)
    for individuo in range(tamanhoPopulacao):
        valorFuncaoObj[individuo] = funcaoObjetivo(populacao[individuo,:], iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)
    return valorFuncaoObj

In [7]:
def selecao(valorFitness):
    pos = np.random.choice(range(tamanhoPopulacao), 2, replace=False)
    valor = valorFitness[pos[0]] < valorFitness[pos[1]]
    if valor == True:
        return pos[0]
    else:
        return pos[1]

In [8]:
def cruzamento(pai1, pai2):
    x = random()
    filho1 = x*pai1+(1-x)*pai2
    filho2 = (1-x)*pai1+x*pai2
    return filho1, filho2

In [9]:
def mutacao(individuo):
    pos = np.random.choice(range(quantidadeVariaveis),  quantidadeTrocas ,replace=False)
    individuo[pos[0]] = individuo[pos[1]]
    individuo[pos[1]] = individuo[pos[0]]
    return individuo

In [10]:
import matplotlib.pyplot as plt

In [11]:
def main(xMin, xMax, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise):
    pop = gerarPopulacao(xMin, xMax)
    fo = calcularFuncao(pop, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)
    fit = fitness(fo)
    melhor_fo = np.zeros(numeroGeracoes)
    melhor_ind_geracao = np.zeros((numeroGeracoes, quantidadeVariaveis))
    for geracao in range(numeroGeracoes):
        new_pop = np.empty((0, quantidadeVariaveis))
        for i in range(round((tamanhoPopulacao) / 2)):
            pai1 = 0
            pai2 = 0
            while pai1 == pai2:
                pai1 = selecao(fit)
                pai2 = selecao(fit)
            if random() < taxaCrossover:
                filho1, filho2 = cruzamento(pop[pai1, :], pop[pai2, :]) 
                if random() < taxaMutacao:    
                    filho1 = mutacao(filho1)
                    filho2 = mutacao(filho2)  
                new_pop = np.vstack([new_pop, filho1, filho2])     
            else:
                pais = np.vstack([pop[pai1, :], pop[pai2, :]])
                new_pop = np.vstack([new_pop, pais])
        pop = new_pop    
        fo = calcularFuncao(pop, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)
        fit = fitness(fo)
        min_fo = min(fo)
        melhor_fo[geracao] = min_fo
        pos_min_fo = np.where(fo == min_fo)

    
    #plt.plot(melhor_fo)
    #plt.xlim(0, numeroGeracoes)
    #plt.ylabel('Gerações')
    #plt.ylabel('Melhor Custo')
    #plt.title('Evolução da função objetivo - GA')
    #plt.grid(True)
    #plt.show()
    

    return pop[pos_min_fo,:][0,0]

In [12]:
# Função de origem e destino
ondaOrigem = [[complex(0.80,0.0)],[complex(0.60,0.0)]]
ondaDestino = [[complex(1/np.sqrt(2),0)],[complex(1/np.sqrt(2),0)]]

hamiltoniano = np.zeros((2,2), dtype=np.complex_) # Hamiltoniano
A = (1/2*np.pi)
B = (3/2*np.pi)
hamiltoniano = [[A, 0], [0, B]]

iteracao = 0
fatorRungeKutta = 0.01
tempoAnalise = 2000
horizonte = 3

xMin = -10
xMax = 10

constantes4 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar se as integrais estão resultando em 1

minimo = main(xMin, xMax, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)

In [13]:
minimo

array([-0.16301078,  0.55843855,  0.61860524])

In [ ]:
while iteracao < 2000:
    controle = np.zeros((2,1), dtype=np.complex_)
    controle = [[0, minimo[0]],[minimo[0],0]]
    hamiltoniano = somar(hamiltoniano, controle)

    constantes4[0][iteracao] = ondaOrigem[0][0] / np.exp(-1*complex(0,1)*(1/2*np.pi)*iteracao/100)
    constantes4[1][iteracao] = ondaOrigem[1][0] / np.exp(-3*complex(0,1)*(1/2*np.pi)*iteracao/100)

    print(constantes4[0][iteracao])
    print(constantes4[1][iteracao])
    print("--------------")
    
    ## Adaptação da onda ao controle

    ondaAvancada = np.zeros((2,2), dtype=np.complex_)
    ondaAvancada = rungeKutta(ondaOrigem, fatorRungeKutta, hamiltoniano, tempoFinal = 0.02, tempoInicial = 0)

    ondaOrigem[0][0] = ondaAvancada[0][0]
    ondaOrigem[1][0] = ondaAvancada[1][0]

    xMax = -minimo[0]
    xMin = minimo[0]
    
    iteracao += 1

    minimo = main(xMax, xMin, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)

(0.8+0j)
(0.6+0j)
--------------
(0.8000142993210706+0.0009779144440106066j)
(0.5999787200027504+0.001303863892156386j)
--------------
(0.8000180143678851+0.0010606755215634004j)
(0.5999733757318503+0.00141420119863147j)
--------------
(0.7999703947961092+0.0004424159655115943j)
(0.6000390182475375+0.0005900307227587634j)
--------------
(0.799848859460208-0.0006560385637744779j)
(0.6002004724848783-0.0008742059516184737j)
--------------
(0.7996313006092807-0.002151242606094717j)
(0.6004805846726614-0.0028674688359159196j)
--------------
(0.799303225658551-0.003958557043862768j)
(0.6008916973460636-0.005277402011537317j)
--------------
(0.7988620332324139-0.005976710283867737j)
(0.6014318035916264-0.00796969144424457j)
--------------
(0.7983050913890689-0.008146329778602739j)
(0.6021001138769709-0.01086603542399768j)
--------------
(0.7976388554543894-0.010398866447028669j)
(0.602886040499349-0.013875929616468233j)
--------------
(0.7968650063628052-0.012703064635625514j)
(0.60378572114

(0.6192138501071952+0.002795643269347127j)
(0.7831938237091006-0.05633666711670154j)
--------------
(0.6178119766542326+0.005962655874313005j)
(0.7844524253019746-0.05391840073820958j)
--------------
(0.6164982699524563+0.00917221236086435j)
(0.7856191913198154-0.051460985978433564j)
--------------
(0.6152740473726598+0.012421459602976767j)
(0.7866929154822249-0.04896737487295544j)
--------------
(0.6141405342663419+0.01570750375914383j)
(0.7876724866481584-0.04644055656563473j)
--------------
(0.6130988627092867+0.019027413239452083j)
(0.7885568899552517-0.043883554243496235j)
--------------
(0.6121500703407884+0.022378221711361983j)
(0.7893452078604277-0.041299422036703635j)
--------------
(0.6112950992997056+0.025756931141113396j)
(0.790036621080713-0.03869124188681168j)
--------------
(0.6105347952584375+0.029160514867634976j)
(0.790630409433306-0.03606212038650183j)
--------------
(0.6098699065558166+0.032585920705808j)
(0.7911259525740271-0.033415185594034856j)
--------------
(0.

(0.755369394666569+0.16106592408091294j)
(0.6346241883409852+0.02696241638539405j)
--------------
(0.7578091303902565+0.15970423730431937j)
(0.6321501872550256+0.024617184657317426j)
--------------
(0.7601901209274423+0.15826371738035114j)
(0.6297388000441084+0.022194728791377957j)
--------------
(0.7625098732436719+0.15674627714889214j)
(0.6273924675845072+0.019697101436124725j)
--------------
(0.7647659568314692+0.15515390828959835j)
(0.6251135637861475+0.017126432337426018j)
--------------
(0.7669560062025086+0.1534886792795435j)
(0.6229043931585001+0.014484926154878484j)
--------------
(0.769077723313116+0.15175273327212047j)
(0.620767188447635+0.011774860201380622j)
--------------
(0.7711288799206161+0.14994828589937131j)
(0.6187041083468598+0.00899858210822386j)
--------------
(0.7731073198681084+0.14807762300000185j)
(0.6167172352832878+0.006158507418081445j)
--------------
(0.7750109612953285+0.1461430982754075j)
(0.6148085732826073+0.0032571171083534568j)
--------------
(0.776

(0.6840621532409649+0.022610016372513125j)
(0.7062711483804358-0.1809110348545066j)
--------------
(0.6809554328749873+0.023456053578003733j)
(0.7092774256781895-0.18076234462608584j)
--------------
(0.6778635849605381+0.024399009905997533j)
(0.712263715757283-0.18051600332805404j)
--------------
(0.6747897062355329+0.025438351116686922j)
(0.7152269032503624-0.18017269265571767j)
--------------
(0.6717368747504796+0.026573444096135768j)
(0.718163896218262-0.17973319370071014j)
--------------
(0.6687081467123892+0.0278035575052927j)
(0.7210716293203838-0.1791983861499282j)
--------------
(0.6657065533510819+0.029127862529718135j)
(0.7239470669578-0.1785692473833833j)
--------------
(0.6627350978111062+0.030545433729261863j)
(0.726787206385866-0.17784685147186552j)
--------------
(0.6597967520724544+0.03205524998681449j)
(0.7295890807931397-0.17703236807548253j)
--------------
(0.656894453903234+0.03365619555516386j)
(0.7323497623434593-0.17612706124417188j)
--------------
(0.65403110384

(0.6579698080329025+0.24346751963023797j)
(0.7120905586438441-0.026950101954648836j)
--------------
(0.66079279062282+0.2448289020619553j)
(0.7090184985922453-0.026541255657951252j)
--------------
(0.6636442312160168+0.24609638638942008j)
(0.7059213310072057-0.026228807629812705j)
--------------
(0.6665211105816193+0.2472691013066318j)
(0.7028021564805756-0.026013504074602782j)
--------------
(0.6694203831449345+0.24834627262258452j)
(0.6996640979109513-0.0258959915427175j)
--------------
(0.6723389801103047+0.24932722405492597j)
(0.6965102973015995-0.025876816267489087j)
--------------
(0.6752738126077689+0.25021137792324355j)
(0.6933439125387929-0.02595642360493079j)
--------------
(0.6782217748603109+0.25099825574122986j)
(0.6901681141538597-0.026135157576937337j)
--------------
(0.6811797473684409+0.25168747870715596j)
(0.6869860820722666-0.026413260518370157j)
--------------
(0.6841446001088496+0.2522787680921147j)
(0.683801002353073-0.026790872828428278j)
--------------
(0.687113

...

(0.5794765390221176+0.25397377820969347j)
(0.7591547800610315-0.15293214219463352j)
--------------
(0.5804054047008451+0.2572542422571921j)
(0.7578310250221454-0.15047231628627755j)
--------------
(0.5814255006113517+0.2604990909932498j)
(0.7564158909025988-0.14805259254582712j)
--------------
(0.5825356422268407+0.26370541365905414j)
(0.7549107106603702-0.14567586804458604j)
--------------
(0.5837345524384242+0.26687033891483875j)
(0.7533169093309988-0.14334499569234774j)
--------------
(0.5850208628646772+0.26999103778388583j)
(0.751636002566452-0.14106278131195926j)
--------------
(0.5863931152547713+0.27306472655310743j)
(0.7498695950810774-0.1388319807621797j)
--------------
(0.5878497629837668+0.2760886696271989j)
(0.7480193790062284-0.13665529711179727j)
--------------
(0.5893891726385094+0.2790601823334713j)
(0.746087132155264-0.1345353778678206j)
--------------
(0.5910096256925288+0.28197663367445j)
(0.7440747162006865-0.13247481226069577j)
--------------
(0.5927093202681959+0

(0.7526612118088319+0.2668361592264174j)
(0.5616553566858538-0.2164319231855494j)
--------------
(0.7528031709335176+0.2641848370338694j)
(0.561356615236201-0.21993740214746485j)
--------------
(0.7528470756217417+0.26152546181677117j)
(0.5611571591328466-0.2234464339417746j)
--------------
(0.7527927212645196+0.2588611691865142j)
(0.5610570537853665-0.22695582996102526j)
--------------
(0.7526400037560048+0.2561950996997207j)
(0.5610562624844138-0.23046240113981956j)
--------------
(0.752388919600301+0.2535303956378761j)
(0.5611546464394036-0.23396296122462895j)
--------------
(0.7520395659150989+0.25087019778521114j)
(0.5613519649208829-0.237454330040693j)
--------------
(0.7515921403321597+0.24821764220809922j)
(0.5616478755074085-0.24093333675271625j)
--------------
(0.7510469407947256+0.2455758570393117j)
(0.5620419344367136-0.24439682311595815j)
--------------
(0.7504043652520829+0.24294795927038731j)
(0.5625335970607678-0.24784164671444334j)
--------------
(0.7496649112515762+0.

(0.5628299595493682+0.23795429854719496j)
(0.7306196217804348-0.3046228858851263j)
--------------
(0.5610554059573479+0.24095928712714718j)
(0.731966579751707-0.3022917884062288j)
--------------
(0.5593651445383664+0.2440140234080435j)
(0.733225340368416-0.2999138833640144j)
--------------
(0.5577607282145952+0.2471157753052183j)
(0.7343944531628637-0.29749201077768767j)
--------------
(0.5562436216431337+0.25026176241655523j)
(0.7354725593451205-0.2950290556521048j)
--------------
(0.5548151997142732+0.25344915887325364j)
(0.7364583931963374-0.29252794501944673j)
--------------
(0.5534767461418065+0.2566750962372159j)
(0.7373507833666177-0.2899916449378261j)
--------------
(0.5522294521468158+0.2599366664420998j)
(0.7381486540761436-0.28742315744982505j)
--------------
(0.5510744152363111+0.2632309247749862j)
(0.738851026218292-0.2848255175041896j)
--------------
(0.5500126380779515+0.2665548928956335j)
(0.7394570183636412-0.28220178984373767j)
--------------
(0.5490450274720188+0.269

(0.6784695332226368+0.40298731831661916j)
(0.5776857074055434-0.2087090710104321j)
--------------
(0.6808541080856491+0.4017139715364519j)
(0.5749697372999181-0.21088695735310187j)
--------------
(0.683184160859852+0.40035875596954096j)
(0.5723122829209469-0.2131451585719723j)
--------------
(0.6854571020386869+0.39892345163033727j)
(0.569715889079673-0.21548175233502584j)
--------------
(0.687670400438832+0.39740992053167884j)
(0.567183037739452-0.21789473582015967j)
--------------
(0.6898215857956622+0.39582010477540264j)
(0.5647161454711137-0.22038202776864485j)
--------------
(0.6919082512962366+0.3941560245608464j)
(0.5623175609751573-0.2229414706190498j)
--------------
(0.6939280560472266+0.3924197761132698j)
(0.559989562673553-0.2255708327193664j)
--------------
(0.6958787274752558+0.3906135295343192j)
(0.5577343563736054-0.22826781061510623j)
--------------
(0.6977580636571865+0.38873952657674044j)
(0.5555540730062554-0.23103003141105957j)
--------------
(0.6995639355779628+0.3

(0.597850256386357+0.2587116356809126j)
(0.6340200563702779-0.4167275618895722j)
--------------
(0.5946055971392796+0.2595417395066833j)
(0.6368762532170282-0.41649830675272265j)
--------------
(0.5913769832903667+0.2604684906003481j)
(0.6397113407976048-0.4161715530257785j)
--------------
(0.5881675051213161+0.26149131828616157j)
(0.6425222120495642-0.41574801908908615j)
--------------
(0.5849802330625572+0.26260955324137264j)
(0.6453057845084235-0.415228522436356j)
--------------
(0.5818182145450305+0.263822428182375j)
(0.6480590034684032-0.41461397883625867j)
--------------
(0.5786844708755439+0.2651290786513202j)
(0.6507788451146589-0.41390540139332066j)
--------------
(0.5755819941389232+0.2665285439023522j)
(0.6534623196238727-0.41310389950897014j)
--------------
(0.5725137441301194+0.2680197678865887j)
(0.656106474229988-0.41221067774384323j)
--------------
(0.5694826453194292+0.2696016003348079j)
(0.6587083962519219-0.41122703458256044j)
--------------
(0.5664915838539348+0.271